# FAST-UAV - Performance Analysis

The analysis mode of FAST-UAV allows to compute the performance of a given design on an operational mission.

In [ ]:
import os.path as pth
import logging
import shutil
import matplotlib.pyplot as plt
import fastoad.api as oad
from fastuav.utils.postprocessing.analysis_and_plots import *

plt.rcParams["figure.figsize"] = 16, 8
plt.rcParams.update({"font.size": 13})

DATA_FOLDER_PATH = "../data"
CONFIGURATIONS_FOLDER_PATH = "../configurations"
WORK_FOLDER_PATH = "./workdir"

CONFIGURATION_FILE = pth.join(WORK_FOLDER_PATH, "multirotor_performance.yaml")
SOURCE_FILE = pth.join(DATA_FOLDER_PATH, "problem_outputs_DJI_M600_mdo.xml")

# For having log messages display on screen
logging.basicConfig(level=logging.INFO, format="%(levelname)-8s: %(message)s")

# For using all screen width
from IPython.core.display import display, HTML

display(HTML("<style>.container { width:95% !important; }</style>"))

In [ ]:
oad.generate_configuration_file(
    CONFIGURATION_FILE, overwrite=True, distribution_name="fastuav", sample_file_name="multirotor_performance.yaml"
)

## 1. Setting up the performance analysis

The model used for the performance analysis solely consists of the `Missions` module. No design models for the components is used, only the performance models.<br>
You can include several missions to be computed:

```yaml
model:
    Performance:
        design_mission:
            id: mission.design
        operational_mission:
           id: mission.concordia
```

Note that the number and types of phases and flight segments are hard-coded for the moment. A future version of FAST-UAV should allow to define the operational mission in a separate .yaml file (see FAST-OAD [mission file](https://fast-oad.readthedocs.io/en/v1.2.1/documentation/mission_module/mission_file/mission_file.html)).

In [ ]:
N2_FILE = pth.join(WORK_FOLDER_PATH, "n2.html")
oad.write_n2(CONFIGURATION_FILE, N2_FILE, overwrite=True)
from IPython.display import IFrame
IFrame(src=N2_FILE, width="100%", height="500px")

## 2. Run the performance analysis

To run a performance analysis, you need an XML file describing the drone design. This can be the output of a design problem (see the [Multirotor Design Notebook](./01_Multirotor_Design.ipynb)).

In [ ]:
oad.generate_inputs(CONFIGURATION_FILE, SOURCE_FILE, overwrite=True)

You can use the `VariableViewer` tool on the input file to fill in the values for the mission profile, in the 'mission' menu (do not forget to save!):

In [ ]:
INPUT_FILE = pth.join(WORK_FOLDER_PATH, "problem_inputs.xml")
oad.variable_viewer(INPUT_FILE)

We now run the problem once (no optimizer is required) using `evaluate_problem`.

In [ ]:
eval_problem = oad.evaluate_problem(CONFIGURATION_FILE, overwrite=True)

In [ ]:
OUTPUT_FILE = pth.join(WORK_FOLDER_PATH, "problem_outputs.xml")
oad.variable_viewer(OUTPUT_FILE)

## 3. Analysis and plots

For the moment, the post processing plots are limited to the visualization of the energy consumed over the mission. If several mission have been computed, you can compare them on the same plot.

In [ ]:
fig = energy_breakdown_sun_plot_drone(OUTPUT_FILE, mission_name="design_mission")
fig = energy_breakdown_sun_plot_drone(OUTPUT_FILE, mission_name="concordia_study", fig=fig)
fig.show()